## Used for selecting the 6 best features per cluster
* We're using mean squared error of each variable vs. the ALSFRS_score, and take the best 6. 

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import pickle
from sklearn import linear_model
from IPython.display import display

from modeling_funcs import *


In [2]:
vectorized_data = pd.read_csv('../all_data_vectorized.csv', sep='|', index_col=0)
slope = pd.read_csv('../all_slope.csv', sep = '|', index_col=0)
clusters = pd.read_csv('../all_forest_clusters.csv', sep = '|', index_col=0)
all_feature_metadata = pickle.load( open('../all_feature_metadata.pickle', 'rb') )

everybody = vectorized_data.join(clusters).join(slope)
Y = everybody[['cluster', 'ALSFRS_slope']]
X = everybody.drop('ALSFRS_slope', 1)

In [3]:
best_features_per_cluster = stepwise_best_features_per_cluster(X, Y, all_feature_metadata)
best_features_per_cluster

cluster: 0 with size: (485, 158) with mean target: -1.18070155828 std: 0.720756033302
best we can do with all features: 0.711720444097
adding best family: ('ALSFRS_Total', 0.70906776290651918) time: 2.35199999809
adding best family: ('ALT(SGPT)', 0.70092659717413774) time: 2.92100000381
adding best family: ('Race', 0.69398396683221886) time: 4.21399998665
adding best family: ('pulse', 0.68816489466541197) time: 5.2349998951
adding best family: ('Calcium', 0.6812285457848406) time: 6.15000009537
adding best family: ('CK', 0.67664695433303557) time: 6.87999987602
cluster: 1 with size: (485, 158) with mean target: -0.905632242827 std: 0.625175960311
best we can do with all features: 0.617457285233
adding best family: ('weight', 0.61656582907120572) time: 2.44799995422
adding best family: ('mouth', 0.61156629829467957) time: 17.0390000343
adding best family: ('CK', 0.60503318539450224) time: 25.6459999084
adding best family: ('Creatinine', 0.5997195865562357) time: 34.9859998226
adding bes

C:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:444: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


{0: ['CK', 'pulse', 'Calcium', 'Race', 'ALSFRS_Total', 'ALT(SGPT)'],
 1: ['CK', 'weight', 'Platelets', 'mouth', 'Creatinine', 'Albumin'],
 2: ['Alkaline Phosphatase',
  'Potassium',
  'pulse',
  'Red Blood Cells (RBC)',
  'Blood Urea Nitrogen (BUN)',
  'ALT(SGPT)'],
 3: ['respiratory',
  'weight',
  'bp_diastolic',
  'Bilirubin (Total)',
  'Albumin',
  'ALSFRS_Total'],
 4: ['Hematocrit',
  'Sodium',
  'mouth',
  'onset_delta',
  'Absolute Basophil Count',
  'Creatinine']}

In [6]:
with open("../best_features_per_cluster.pickle", "wb") as output_file:
    pickle.dump(best_features_per_cluster, output_file)


#Apply the selector 
leave only the best features per cluster

In [7]:
for t in ["all", "test"]:
    print t
    df = pd.read_csv('../' + t + '_data.csv', sep = '|', index_col="SubjectID", dtype='unicode')
    print "df", df.shape
    clusters = pd.read_csv('../' + t + '_forest_clusters.csv', sep = '|', index_col="SubjectID")
    print "clusters", clusters.groupby('cluster').size()
    buf = filter_only_selected_features(df, clusters, best_features_per_cluster)
    with open('../' + t + '_data_selected.csv','w') as f:
        f.write(buf)


all
df (1642927, 5)
clusters cluster
0    485
1    485
2    484
3    485
4    485
dtype: int64
test
df (126664, 5)
clusters cluster
0    109
1    128
2    122
3    119
4    122
dtype: int64
